In [1]:
import torch
from torch.utils.tensorboard.summary import image
import torchvision
import torch.nn.functional as F
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim


from torch.utils.tensorboard import SummaryWriter
myWriter = SummaryWriter('./tensorboard/log/')

In [2]:
myTransforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [3]:
#  load
train_dataset = torchvision.datasets.CIFAR10(root='./cifar-10-python/', train=True, download=True, transform=myTransforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)

test_dataset = torchvision.datasets.CIFAR10(root='./cifar-10-python/', train=False, download=True, transform=myTransforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=True, num_workers=0)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python/cifar-10-python.tar.gz to ./cifar-10-python/
Files already downloaded and verified


In [4]:
# 定义模型
myModel = torchvision.models.resnet50(pretrained=True)
# 将原来的ResNet18的最后两层全连接层拿掉,替换成一个输出单元为10的全连接层
inchannel = myModel.fc.in_features
myModel.fc = nn.Linear(inchannel, 10)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [5]:
# 损失函数及优化器
# GPU加速
myDevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(myDevice)

In [6]:
learning_rate=0.001
myOptimzier = optim.SGD(myModel.parameters(), lr = learning_rate, momentum=0.9)
myLoss = torch.nn.CrossEntropyLoss()

In [ ]:
for _epoch in range(10):
    training_loss = 0.0
    for _step, input_data in enumerate(train_loader):
        image, label = input_data[0].to(myDevice), input_data[1].to(myDevice)   # GPU加速
        predict_label = myModel.forward(image)
       
        loss = myLoss(predict_label, label)

        myWriter.add_scalar('training loss', loss, global_step = _epoch*len(train_loader) + _step)

        myOptimzier.zero_grad()
        loss.backward()
        myOptimzier.step()

        training_loss = training_loss + loss.item()
        if _step % 10 == 0 :
            print('[iteration - %3d] training loss: %.3f' % (_epoch*len(train_loader) + _step, training_loss/10))
            training_loss = 0.0
            print()
    correct = 0
    total = 0
    #torch.save(myModel, 'Resnet50_Own.pkl') # 保存整个模型
    myModel.eval()
    for images,labels in test_loader:
        # GPU加速
        images = images.to(myDevice)
        labels = labels.to(myDevice)     
        outputs = myModel(images)   # 在非训练的时候是需要加的，没有这句代码，一些网络层的值会发生变动，不会固定
        numbers,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()

    print('Testing Accuracy : %.3f %%' % ( 100 * correct / total))
    myWriter.add_scalar('test_Accuracy',100 * correct / total)

[iteration -   0] training loss: 0.233

[iteration -  10] training loss: 2.266

[iteration -  20] training loss: 2.101

[iteration -  30] training loss: 1.855

[iteration -  40] training loss: 1.616

[iteration -  50] training loss: 1.341

[iteration -  60] training loss: 1.129

[iteration -  70] training loss: 0.953

[iteration -  80] training loss: 0.790

[iteration -  90] training loss: 0.717

[iteration - 100] training loss: 0.648

[iteration - 110] training loss: 0.614

[iteration - 120] training loss: 0.543

[iteration - 130] training loss: 0.506

